In [3]:
# %load XGboost.py
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import metrics
from matplotlib import pyplot as plt


df = pd.read_csv("train.csv")
df["time"] = df["date"].apply(lambda x : int(x[-5:-3]))
df["day"] = df["date"].apply(lambda x : int(x.split('/')[0]))
df["month"] = df["date"].apply(lambda x : int(x.split('/')[1]))
df["year"] = df["date"].apply(lambda x : int(x.split('/')[2][0:4]))

########################################################holiday
# df['holiday'] = 0
# for i in range(14006):
#     if df["month"][i] == "1" and df['year'][i] == '2017' and df["day"][i] in ['2', '28', '30', '31']:
#         df['holiday'][i] = 1
#     elif df["month"][i] == "4" and df['year'][i] == '2017' and df["day"][i] in ['4', '14','15','17']:
#         df['holiday'][i] = 1
#     elif df["month"][i] == "4" and df['year'][i] == '2017' and df["day"][i] in ['2', '3', '4', '29', '30']:
#         df['holiday'][i] = 1
#     elif df["month"][i] == "5" and df['year'][i] == '2017' and df["day"][i] in ['1', '3',  '30']:
#         df['holiday'][i] = 1
#     elif df["month"][i] == "7" and df['year'][i] == '2017' and df["day"][i] in ['1']:
#         df['holiday'][i] = 1
#     elif df["month"][i] == "10" and df['year'][i] == '2017' and df["day"][i] in [ '2', '5', '28']:
#         df['holiday'][i] = 1
#     elif df["month"][i] == "12" and df['year'][i] == '2017' and df["day"][i] in ['25','26']:
#         df['holiday'][i] = 1
        
#     elif dft["month"][i] == "1" and dft['year'][i] == '2018' and dft["day"][i] in ['1']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "2" and dft['year'][i] == '2018' and dft["day"][i] in ['16', '17', '19']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "3" and dft['year'][i] == '2018' and dft["day"][i] in ['30', '31']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "4" and dft['year'][i] == '2018' and dft["day"][i] in ['2', '5']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "5" and dft['year'][i] == '2018' and dft["day"][i] in ['1','22']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "6" and dft['year'][i] == '2018' and dft["day"][i] in [ '18']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "7" and dft['year'][i] == '2018' and dft["day"][i] in ['2']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "9" and dft['year'][i] == '2018' and dft["day"][i] in ['25']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "10" and dft['year'][i] == '2018' and dft["day"][i] in ['1', '17']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "12" and dft['year'][i] == '2018' and dft["day"][i] in ['25', '26']:
#         dft['holiday'][i] = 1

###########################################################################seasons
df['spring'] = 0
df['summer'] = 0
df['autumn'] = 0
df['winter'] = 0
df['winter'].iloc[0:783] = 1
df['spring'].iloc[783:2967] = 1
df['summer'].iloc[2967:5222] = 1
df['autumn'].iloc[5222:7430] = 1
df['winter'].iloc[7430:9246] = 1
df['spring'].iloc[9246:10566] = 1
df['summer'].iloc[10566:11975] = 1
df['autumn'].iloc[11975:13260] = 1
df['winter'].iloc[13260:14006] = 1
######################################################################################

df.drop(["date"], axis = 1, inplace=True)
df.drop(["id"], axis = 1, inplace=True)
ydata = df["speed"]
xdata = df[["time", "day", "month", 'spring', 'summer', 'autumn', 'winter']]
xdata['year'] = 0
xdata['year'].iloc[8750:14006] = 1
##############################(weekend)
day = df["day"]
weekend = [7]
dayindex = 1
for i in range(1, len(day)):
    if int(day[i-1]) == int(day[i]):
        weekend.append(weekend[-1])
    else:
        dayindex += 1
        if dayindex%7 == 2:
            weekend.append(1)
        elif dayindex%7 == 3:
            weekend.append(2)
        elif dayindex%7 == 4:
            weekend.append(3)
        elif dayindex%7 == 5:
            weekend.append(4)
        elif dayindex%7 == 6:
            weekend.append(5)
        elif dayindex%7 == 0:
            weekend.append(6)
        else: weekend.append(7)
xdata["weekend"] = weekend



##############################################################testdata
dft = pd.read_csv("test.csv")
dft["time"] = dft["date"].apply(lambda x : int(x[-5:-3]))
dft["day"] = dft["date"].apply(lambda x : int(x.split('/')[0]))
dft["month"] = dft["date"].apply(lambda x : int(x.split('/')[1]))
dft["year"] = dft["date"].apply(lambda x : int(x.split('/')[2][0:4]))
dayt = dft["day"]

#################################################################holiday
# dft['holiday'] = 0
# for i in range(3504):
#     if dft["month"][i] == "1" and dft['year'][i] == '2018' and dft["day"][i] in ['1']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "2" and dft['year'][i] == '2018' and dft["day"][i] in ['16', '17', '19']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "3" and dft['year'][i] == '2018' and dft["day"][i] in ['30', '31']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "4" and dft['year'][i] == '2018' and dft["day"][i] in ['2', '5']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "5" and dft['year'][i] == '2018' and dft["day"][i] in ['1','22']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "6" and dft['year'][i] == '2018' and dft["day"][i] in [ '18']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "7" and dft['year'][i] == '2018' and dft["day"][i] in ['2']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "9" and dft['year'][i] == '2018' and dft["day"][i] in ['25']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "10" and dft['year'][i] == '2018' and dft["day"][i] in ['1', '17']:
#         dft['holiday'][i] = 1
#     elif dft["month"][i] == "12" and dft['year'][i] == '2018' and dft["day"][i] in ['25', '26']:
#         dft['holiday'][i] = 1

#####################################################################seasons
dft['spring'] = 0
dft['summer'] = 0
dft['autumn'] = 0
dft['winter'] = 0
dft['winter'].iloc[0:320] = 1
dft['spring'].iloc[320:1160] = 1
dft['summer'].iloc[1160:2007] = 1
dft['autumn'].iloc[2007:2930] = 1
dft['winter'].iloc[2930:3504] = 1

###############################################################################

dft.drop(["date"], axis = 1, inplace=True)
dft.drop(["id"], axis = 1, inplace=True)

# tempdf = pd.get_dummies(dft[["time", "day", "month"]])
# temp = tempdf["year_2018"]
# del tempdf["year_2018"]
# tempdf["year_2017"] = 0
# tempdf = pd.concat([tempdf, temp], axis = 1)

xtest = dft[["time", "day", "month", 'spring', 'summer', 'autumn', 'winter']]
xtest['year'] = 1

##########################################################################weekend
weekendt = [1]
daytindex = 1
for i in range(1, len(dayt)):
    if int(dayt[i-1]) == int(dayt[i]):
        weekendt.append(weekendt[-1])
    else:
        daytindex += 1
        if daytindex%7 == 1:
            weekendt.append(1)
        elif daytindex%7 == 2:
            weekendt.append(2)
        elif daytindex%7 == 3:
            weekendt.append(3)
        elif daytindex%7 == 4:
            weekendt.append(4)
        elif daytindex%7 == 5:
            weekendt.append(5)
        elif daytindex%7 == 6:
            weekendt.append(6)
        else: weekendt.append(7)
xtest["weekend"] = weekendt



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52146)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.0.1/libexec/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.0.1/libexec/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52146)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.0.1/libexec/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError:

In [4]:
traindate = pd.read_csv('./train.csv',parse_dates=['date'])
testdate = pd.read_csv('./test.csv',parse_dates=['date'],index_col = 0)
trdate=traindate['date']
tedate=testdate['date']
xd=pd.concat([xdata,trdate], axis=1)
xt=pd.concat([xtest,tedate], axis=1)

In [5]:
public_vacation_list = [
    '20180101', '20180216','20180217','20180219','20180330','20180331','20180402','20180405',
    '20180501','20180522','20180618','20180702','20180925','20181001','20181017','20181225','20181226',
    '20170102','20170128','20170130','20170131','20170404','20170414','20170415','20170417','20170501',
    '20170503','20170530','20170701','20171002','20171005','20171028','20171225','20171226'
] 
for df in (xd,xt):
    df['only-ymd'] = df['date'].apply(lambda x: x.strftime('%Y%m%d'))
    df['y/n-holiday'] = df['only-ymd'].apply(lambda x: 1 if x in public_vacation_list else 0)

In [6]:
xdata=xd.drop(['date','only-ymd'],axis=1)
xtest=xt.drop(['date','only-ymd'],axis=1)

In [7]:
xdata

,time,day,month,spring,summer,autumn,winter,year,weekend,y/n-holiday
0,0,1,1,0,0,0,1,0,7,0
1,1,1,1,0,0,0,1,0,7,0
2,2,1,1,0,0,0,1,0,7,0
3,3,1,1,0,0,0,1,0,7,0
4,4,1,1,0,0,0,1,0,7,0
...,...,...,...,...,...,...,...,...,...,...
14001,12,31,12,0,0,0,1,1,1,0
14002,15,31,12,0,0,0,1,1,1,0
14003,16,31,12,0,0,0,1,1,1,0
14004,18,31,12,0,0,0,1,1,1,0


In [13]:
xdata17=xdata.iloc[0:8750]
xdata17

,time,day,month,spring,summer,autumn,winter,year,weekend,y/n-holiday
0,0,1,1,0,0,0,1,0,7,0
1,1,1,1,0,0,0,1,0,7,0
2,2,1,1,0,0,0,1,0,7,0
3,3,1,1,0,0,0,1,0,7,0
4,4,1,1,0,0,0,1,0,7,0
...,...,...,...,...,...,...,...,...,...,...
8745,19,31,12,0,0,0,1,0,7,0
8746,20,31,12,0,0,0,1,0,7,0
8747,21,31,12,0,0,0,1,0,7,0
8748,22,31,12,0,0,0,1,0,7,0


In [17]:
ydata17=ydata.iloc[0:8750]
ydata17

0       43.002930
1       46.118696
2       44.294158
3       41.067468
4       46.448653
          ...    
8745    44.742372
8746    46.428155
8747    43.059179
8748    42.754849
8749    45.894518
Name: speed, Length: 8750, dtype: float64

In [14]:
xdata18=xdata.iloc[8750:14006]
xdata18

,time,day,month,spring,summer,autumn,winter,year,weekend,y/n-holiday
8750,0,1,1,0,0,0,1,1,1,1
8751,1,1,1,0,0,0,1,1,1,1
8752,3,1,1,0,0,0,1,1,1,1
8753,4,1,1,0,0,0,1,1,1,1
8754,6,1,1,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
14001,12,31,12,0,0,0,1,1,1,0
14002,15,31,12,0,0,0,1,1,1,0
14003,16,31,12,0,0,0,1,1,1,0
14004,18,31,12,0,0,0,1,1,1,0


In [18]:
ydata18=ydata.iloc[8750:14006]
ydata18

8750     43.707252
8751     47.648155
8752     50.626977
8753     48.740828
8754     46.179884
           ...    
14001    19.865269
14002    17.820375
14003    12.501851
14004    15.979319
14005    40.594183
Name: speed, Length: 5256, dtype: float64

In [70]:
##################################################################

x_train, x_test, y_train, y_test = train_test_split(
    xdata17, ydata17, test_size=0.2, random_state=1)

params = {
    'booster': 'gbtree',
    'objective': 'reg:squarederror',
    'gamma': 0.3,
    'max_depth': 60,
    'lambda': 5,
    'subsample': 1,
    'colsample_bytree': 1,
    'min_child_weight': 0,
    'silent': 0,
    'eta': 0.01,
    'seed': 0,
    'nthread': 4,
    'reg_alpha': 1,

    
}

dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_test, y_test)
num_rounds = 650
plst = list(params.items())
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(plst, dtrain, num_rounds)

################################################################################
predict_y_train = model.predict(xgb.DMatrix(x_train))
predict_y_test = model.predict(xgb.DMatrix(x_test))
print("trainMSE:",metrics.mean_squared_error(y_train, predict_y_train))
print("testMSE:",metrics.mean_squared_error(y_test, predict_y_test))

[00:50:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


trainMSE: 2.118558123068144
testMSE: 10.924367223660067


In [71]:
predict_y_test = model.predict(xgb.DMatrix(xdata18))

print("testMSE:",metrics.mean_squared_error(ydata18, predict_y_test))

testMSE: 19.090731196681052
